In [ ]:
def live_location():
    current_location=busestrams_get()
    filtered=current_location[current_location['Lines']=='523']
    lat=current_location['Lat'][1]
    lon=current_location['Lon'][1]
    time=current_location['Time'][1][11:20]
    return lat,lon,time

def stop_location():
    bus_stop_location=dbstore_get()
    bus_stop=bus_stop_location[bus_stop_location['kierunek']=='PKP Olszynka Grochowska']
    bus_stop=bus_stop[['dlug_geo','szer_geo']]
    bus_stop=bus_stop.iloc[0,:]
    bus_stop['lon']=bus_stop['dlug_geo']
    bus_stop['lat']=bus_stop['szer_geo']
    bus_stop=bus_stop.drop(['dlug_geo','szer_geo'])
    stop_lon=bus_stop['lon']
    stop_lat=bus_stop['lat']
    return stop_lon,stop_lat

def coord_distance(coord1, coord2):

    lon1 = radians(coord1[1])
    lon2 = radians(coord2[1])
    lat1 = radians(coord1[0])
    lat2 = radians(coord2[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2

    c = 2 * asin(sqrt(a))

    # Radius of earth in meters. Use 3956 for miles
    r = 6371000

    return (c * r)


def bus_at_stop(line):
    lat,lon,time=live_location()
    stop_lon,stop_lat=stop_location()
    lon_low=stop_lon - 0.0002
    lon_high=stop_lon + 0.0002
    lat_low=stop_lat - 0.0002
    lat_high=stop_lat + 0.0002
    if (lat > lat_low & lat < lat_high) & (lon > long_low & lon < lon_high):
        arrival_time=time
    else:
        new_lat,new_lon,new_time=live_location()
        if (new_lat > lat_low & new_lat < lat_high) & (new_lon > long_low & new_lon < lon_high):
            arrival_time=time
        elif (lat < stop_lat & new_lat > stop_lat) or (lat > stop_lat & new_lat < stop_lat) or (lon < stop_lon & new_lon > lon ) or (lon > stop_lon & new_lon < lon):
            distance=coord_distance((lat,lon),(new_lat,new_lon))
            speed=distance/10
            distance_till_center=coord_distance((lat,lon),(stop_lon,stop_lat))
            time_till_center=distance_till_center/speed
            time=time+time_till_center
        else:
            print('Bus to far from the stop')